<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/%D0%94%D0%97_%D1%87%D0%B0%D1%81%D1%82%D1%8C_1_ver_2_%D1%80%D0%B0%D0%B1%D0%BE%D1%87%D0%B0%D1%8F_%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader

In [2]:
# Параметры
ALPHABET = 'abcdefghijklmnopqrstuvwxyz '  # 26 букв + пробел
MAX_LENGTH = 30  # Максимальная длина предложения
VOCAB_SIZE = len(ALPHABET)
EMBEDDING_DIM = 32
LSTM_UNITS = 128
BATCH_SIZE = 64
NUM_EPOCHS = 20

In [3]:
# Создание словарей для преобразования символов
char_to_idx = {char: idx for idx, char in enumerate(ALPHABET)}
idx_to_char = {idx: char for idx, char in enumerate(ALPHABET)}

In [4]:
# Функция шифрования Цезаря
def caesar_encrypt(text, shift):
    encrypted = []
    for char in text.lower():
        if char not in ALPHABET:
            continue
        if char == ' ':
            encrypted.append(' ')
        else:
            idx = ALPHABET.index(char)
            encrypted_char = ALPHABET[(idx + shift) % 26]
            encrypted.append(encrypted_char)
    return ''.join(encrypted).ljust(MAX_LENGTH, ' ')[:MAX_LENGTH]



In [5]:
# Генератор данных
class CaesarDataset(Dataset):
    def __init__(self, sentences, num_samples=10000):
        self.sentences = sentences
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        orig_text = random.choice(self.sentences).lower()[:MAX_LENGTH]
        shift = random.randint(1, 25)

        encrypted = caesar_encrypt(orig_text, shift)
        orig_padded = orig_text.ljust(MAX_LENGTH, ' ')[:MAX_LENGTH]

        # Преобразование в индексы
        X = [char_to_idx[c] for c in encrypted]
        y = [char_to_idx[c] for c in orig_padded]

        return torch.tensor(X), torch.tensor(y)

In [6]:
# Модель нейронной сети
class DecryptModel(nn.Module):
    def __init__(self):
        super(DecryptModel, self).__init__()
        self.embedding = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        self.lstm = nn.LSTM(
            input_size=EMBEDDING_DIM,
            hidden_size=LSTM_UNITS,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )
        self.fc = nn.Linear(LSTM_UNITS, VOCAB_SIZE)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out)
        return output

In [7]:
# Функция для декодирования вывода модели
def decode_output(output):
    indices = torch.argmax(output, dim=-1).squeeze(0).cpu().numpy()
    return ''.join(idx_to_char[idx] for idx in indices)

In [8]:
 # Создание датасетов и загрузчиков
sentences = [
        "hello world",
        "neural networks",
        "deep learning",
        "caesar cipher",
        "secret message",
        "artificial intelligence",
        "machine learning model",
        "decode this text",
        "example sentence",
        "natural language processing",
        "the quick brown fox jumps over the lazy dog",
        "pyTorch is amazing for deep learning",
        "recurrent neural networks are powerful",
        "text decryption using lstm",
        "caesar shift cipher decoder"
    ]
train_dataset = CaesarDataset(sentences, num_samples=10000)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
    # Инициализация модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DecryptModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Обучение модели
for epoch in range(NUM_EPOCHS):
   total_loss = 0
   for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

            # Изменяем форму для вычисления потерь
        loss = criterion(outputs.view(-1, VOCAB_SIZE), targets.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if i % 50 == 0:
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}')

print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Average Loss: {total_loss/len(train_loader):.4f}')

Epoch [1/20], Step [0/157], Loss: 3.2776
Epoch [1/20], Step [50/157], Loss: 1.8982
Epoch [1/20], Step [100/157], Loss: 1.7377
Epoch [1/20], Step [150/157], Loss: 1.6976
Epoch [2/20], Step [0/157], Loss: 1.7367
Epoch [2/20], Step [50/157], Loss: 1.5219
Epoch [2/20], Step [100/157], Loss: 1.3831
Epoch [2/20], Step [150/157], Loss: 1.2362
Epoch [3/20], Step [0/157], Loss: 1.1350
Epoch [3/20], Step [50/157], Loss: 0.9724
Epoch [3/20], Step [100/157], Loss: 0.7910
Epoch [3/20], Step [150/157], Loss: 0.7050
Epoch [4/20], Step [0/157], Loss: 0.7366
Epoch [4/20], Step [50/157], Loss: 0.6688
Epoch [4/20], Step [100/157], Loss: 0.6376
Epoch [4/20], Step [150/157], Loss: 0.5976
Epoch [5/20], Step [0/157], Loss: 0.6013
Epoch [5/20], Step [50/157], Loss: 0.6043
Epoch [5/20], Step [100/157], Loss: 0.5871
Epoch [5/20], Step [150/157], Loss: 0.5597
Epoch [6/20], Step [0/157], Loss: 0.5333
Epoch [6/20], Step [50/157], Loss: 0.5145
Epoch [6/20], Step [100/157], Loss: 0.4629
Epoch [6/20], Step [150/157],

In [11]:
# Тестирование модели
test_sentences = [
        "hello ai",
        "this is a secret message",
        "neural networks can break ciphers"
    ]

model.eval()
with torch.no_grad():
    for sentence in test_sentences:
        shift = random.randint(1, 25)
        encrypted = caesar_encrypt(sentence, shift)

        input_seq = torch.tensor([char_to_idx[c] for c in encrypted]).unsqueeze(0).to(device)
        output = model(input_seq)

        decrypted = decode_output(output)

        print(f"\nOriginal:  '{sentence}'")
        print(f"Encrypted: '{encrypted.strip()}' (shift: {shift})")
        print(f"Decrypted: '{decrypted.strip()}'")


Original:  'hello ai'
Encrypted: 'byffi uc' (shift: 20)
Decrypted: 'nello wo'

Original:  'this is a secret message'
Encrypted: 'cqrb rb j bnlanc vnbbjpn' (shift: 9)
Decrypted: 'naeh le q qmikii frrresi'

Original:  'neural networks can break ciphers'
Encrypted: 'izpmvg izorjmfn xvi wmzvf xdkc' (shift: 21)
Decrypted: 'drural networks are deepl lerr'
